In [1]:
#!pip install pandas-profiling
#!pip install openpyxl
#!pip install shapely

import pandas as pd
import numpy as np
import pickle
from shapely import wkt
import geopandas as gpd

In [2]:
xls = pd.ExcelFile('../datasets/homicidios.xlsx')
data = pd.read_excel(xls, 'HECHOS')

In [3]:
data.columns= data.columns.str.lower()

In [4]:
data.head(5)

,id,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección normalizada,comuna,xy (caba),pos x,pos y,participantes,victima,acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [5]:
#Verificador de Dimension del DataFrame de Hechos
data.shape

(696, 21)

In [6]:
#Tipo de Datos de Columnas
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     696 non-null    object        
 1   n_victimas             696 non-null    int64         
 2   fecha                  696 non-null    datetime64[ns]
 3   aaaa                   696 non-null    int64         
 4   mm                     696 non-null    int64         
 5   dd                     696 non-null    int64         
 6   hora                   696 non-null    object        
 7   hh                     696 non-null    object        
 8   lugar_del_hecho        696 non-null    object        
 9   tipo_de_calle          696 non-null    object        
 10  calle                  695 non-null    object        
 11  altura                 129 non-null    float64       
 12  cruce                  525 non-null    object        
 13  direc

### Vericando Duplicados

In [7]:
print(f"Numero de filas duplicadas = {data.duplicated().sum()}")

Numero de filas duplicadas = 0


### Verificando Columnas con valores nulos

In [8]:
data.isnull().sum()

id                         0
n_victimas                 0
fecha                      0
aaaa                       0
mm                         0
dd                         0
hora                       0
hh                         0
lugar_del_hecho            0
tipo_de_calle              0
calle                      1
altura                   567
cruce                    171
dirección normalizada      8
comuna                     0
xy (caba)                  0
pos x                      0
pos y                      0
participantes              0
victima                    0
acusado                    0
dtype: int64

Se prescinde de la columna 'altura' ya que los valores nulos representan más del 30% del Total 

In [9]:
#Eliminación de las Columnas 'Altura' 
data.drop(columns=['altura'], inplace=True)

### Conversión de Tipo de Dato

In [10]:
data['fecha'] = data['fecha'].dt.date
data['fecha'] = data['fecha'].replace(pd.NaT, '')

In [11]:
data['hh'] = pd.to_numeric(data['hh'], errors="coerce")

In [12]:
data['pos x'] = pd.to_numeric(data['pos x'], errors="coerce" )
data['pos y'] = pd.to_numeric(data['pos y'], errors="coerce" )

### Reemplazo de Valores 

In [13]:
data['calle'].fillna('SD', inplace=True)
data['cruce'].fillna('SD', inplace=True)
data['dirección normalizada'].fillna('SD', inplace=True)

In [14]:
data['lugar_del_hecho'] = data['lugar_del_hecho'].str.title()
data['tipo_de_calle'] = data['tipo_de_calle'].str.title()
data['calle'] = data['calle'].str.title()

data['cruce'] = data['cruce'].str.title()
data['dirección normalizada'] = data['dirección normalizada'].str.title()
data['participantes'] = data['participantes'].str.title()
data['acusado'] = data['acusado'].str.title()

In [15]:
data['comuna'] = 'comuna ' + data['comuna'].astype(str)

In [16]:
data['comuna'] = data['comuna'].replace('comuna 0', 'SD')

In [17]:
## ADD
data['hh'] = data['hh'].astype('Int64')

Se prescinde de las columnas 'AAAA', 'MM' , 'DD' y 'HORA' las 3 primeras se desprenden de la columa 'FECHA' y la columna 'HORA' es mejor explicada en la columna 'HH'

In [18]:
data.drop(columns=['aaaa'], inplace=True)
data.drop(columns=['mm'], inplace=True)
data.drop(columns=['dd'], inplace=True)
data.drop(columns=['hora'], inplace=True)
data.drop(columns=['victima'], inplace=True)

In [19]:
data.isna().sum()

id                        0
n_victimas                0
fecha                     0
hh                        1
lugar_del_hecho           0
tipo_de_calle             0
calle                     0
cruce                     0
dirección normalizada     0
comuna                    0
xy (caba)                 0
pos x                    12
pos y                    12
participantes             0
acusado                   0
dtype: int64

In [20]:
data.rename(columns={'id': 'id_hecho'}, inplace=True)
data.rename(columns={'dirección normalizada': 'direc_normalizada'}, inplace=True)
data.rename(columns={'lugar_del_hecho': 'lugar_hecho'}, inplace=True)
data.rename(columns={'tipo_de_calle': 'tipo_calle'}, inplace=True)
data.rename(columns={'pos x': 'longitud'}, inplace=True)
data.rename(columns={'pos y': 'latitud'}, inplace=True)

In [21]:
data.rename(columns={'hh': 'franja_horaria'}, inplace=True)
data.rename(columns={'n_victimas': 'nro_victimas'}, inplace=True)
data.rename(columns={'xy (caba)': 'localizacion'}, inplace=True)

In [22]:
# Se elimina esta columna porque se puede hallar en base a la latitud y longitud
#data.drop(columns=['localizacion'], inplace=True)

In [23]:
data.head(5)

,id_hecho,nro_victimas,fecha,franja_horaria,lugar_hecho,tipo_calle,calle,cruce,direc_normalizada,comuna,localizacion,longitud,latitud,participantes,acusado
0,2016-0001,1,2016-01-01,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,Piedra Buena Av.,"Fernandez De La Cruz, F., Gral. Av.","Piedra Buena Av. Y Fernandez De La Cruz, F., G...",comuna 8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,Moto-Auto,Auto
1,2016-0002,1,2016-01-02,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,"Paz, Gral. Av.",De Los Corrales Av.,"Paz, Gral. Av. Y De Los Corrales Av.",comuna 9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,Auto-Pasajeros,Pasajeros
2,2016-0003,1,2016-01-03,7,Av Entre Rios 2034,Avenida,Entre Rios Av.,Sd,Entre Rios Av. 2034,comuna 1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,Moto-Auto,Auto
3,2016-0004,1,2016-01-10,0,Av Larrazabal Y Gral Villegas Conrado,Avenida,Larrazabal Av.,"Villegas, Conrado, Gral.","Larrazabal Av. Y Villegas, Conrado, Gral.",comuna 8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,Moto-Sd,Sd
4,2016-0005,1,2016-01-21,5,Av San Juan Y Presidente Luis Saenz Peña,Avenida,San Juan Av.,"Saenz Pe?A, Luis, Pres.","San Juan Av. Y Saenz Peã‘A, Luis, Pres.",comuna 1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,Moto-Pasajeros,Pasajeros


In [24]:
#data[data['id_hecho']=='2017-0050']

In [25]:
#Consulta duplicado por id_hecho
duplicate = data[data.duplicated('id_hecho')]

In [26]:
duplicate

,id_hecho,nro_victimas,fecha,franja_horaria,lugar_hecho,tipo_calle,calle,cruce,direc_normalizada,comuna,localizacion,longitud,latitud,participantes,acusado


In [27]:

pickle_out = open("../datasets/dataHecho.pkl","wb")
pickle.dump(data, pickle_out)
pickle_out.close()

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_hecho           696 non-null    object 
 1   nro_victimas       696 non-null    int64  
 2   fecha              696 non-null    object 
 3   franja_horaria     695 non-null    Int64  
 4   lugar_hecho        696 non-null    object 
 5   tipo_calle         696 non-null    object 
 6   calle              696 non-null    object 
 7   cruce              696 non-null    object 
 8   direc_normalizada  696 non-null    object 
 9   comuna             696 non-null    object 
 10  localizacion       696 non-null    object 
 11  longitud           684 non-null    float64
 12  latitud            684 non-null    float64
 13  participantes      696 non-null    object 
 14  acusado            696 non-null    object 
dtypes: Int64(1), float64(2), int64(1), object(11)
memory usage: 82.4+ KB


In [29]:
data.insert(0, 'id', range(1, 1 + len(data)))

In [30]:
def getNormalizeDate(text):   
    text = str.lstrip(str.rstrip(text))
    text = text.replace('"', '')

    return text 

In [31]:
data['id_hecho']= data['id_hecho'].astype(str).apply(getNormalizeDate)
data['fecha']=data['fecha'].astype(str).apply(getNormalizeDate)
data['lugar_hecho']=data['lugar_hecho'].astype(str).apply(getNormalizeDate)
data['tipo_calle']=data['tipo_calle'].astype(str).apply(getNormalizeDate)     
data['calle']=data['calle'].astype(str).apply(getNormalizeDate)               
data['cruce']=data['cruce'].astype(str).apply(getNormalizeDate)              
data['direc_normalizada']=data['direc_normalizada'].astype(str).apply(getNormalizeDate)
data['comuna']=data['comuna'].astype(str).apply(getNormalizeDate)
data['participantes']=data['participantes'].astype(str).apply(getNormalizeDate)
data['acusado']=data['acusado'].astype(str).apply(getNormalizeDate)

In [32]:
data.to_csv('../sql/dataHecho.csv', header=False, index=False)